## IBM Applied Data Science Capstone


This notebook mainly deals with the capstone project carried out as a part of IBM Applied Data Science course covering the following topics:

    -FourSquare API
    -Clustering
    -Reporting based on the above results
    

In [1]:
import pandas as pd
import numpy as np

In [2]:
print("Hello Capstone Project Course!")

Hello Capstone Project Course!
